# <span style='color:black'> <center>Portafolios de inversión con Python</center> </span>
## Actuarios por México
<p><img src="https://blog.axend.io/wp-content/uploads/2022/03/portafolio-de-inversion-1024x675.jpg" width="650"</p>

Descargaremos la información de un portafolio compuesto por 5 acciones. Vamos a utilizar Apple, Nike, Google y Micrisoft.\
Utilizaremos una ventana de tiempo del 01/01/2018 a dia de hoy.

In [ ]:
#Si no tenemos instalada la libreria yfinance, ejecutar el siguiente comando
#!pip install yfinance

Como primer paso, debemos importas las librerias necesarias para el análisis.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

Posteriormente, en un Data Frame vamos a almacenar la información de los precios de cierre de las 5 acciones seleccionadas.

In [8]:
tickers = ['AAPL', 'NKE', 'GOOGL', 'MSFT']
#tickers = ['WMT', 'META', 'BP']

portafolio = pd.DataFrame()

for t in tickers:
    portafolio[t] = yf.download(
        t, start = '2018-01-01',end = '2024-10-22')['Close']

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: SSLError(MaxRetryError("HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)')))"))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NKE']: SSLError(MaxRetryError("HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)')))"))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GOOGL']: SSLError(MaxRetryError("HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCer

Visualizamos el contenido del Data Frame que acabamos de construir.

In [ ]:
portafolio.tail()

Realizamos un gráfico de serie para comparar de manera visual el comportamiento de las series importadas.

In [ ]:
portafolio.plot(figsize=(10,6))
plt.title('Portafolio de acciones')
plt.xlabel('Tiempo')
plt.ylabel('Precios')
plt.show()

## Normalizar a 100

Cuando se analiza en conjunto los precios de acciones, es común tener precios que se desenvuelven a diferentes magnitudes. En estos casos, se realiza una técnica conocida como "Normalizar series a 100", la cual consiste en dividir todos los registros de la serie de tiempo entre el primer registro.


$$
\frac {P_t}{P_0} * 100
$$

Esto forzará a que todas las series comiencen desde un mismo punto en común (100). De esta manera, podemos comparar el crecimiento/decremento en los precios de acciones que no tengan necesariamente precios en los mismos rangos.

In [ ]:
portafolio_norm = (portafolio/portafolio.iloc[0]*100)
portafolio_norm.head(10)

Volvemos a realizar nuestro gráfico ahora con las series normalizadas a 100.

In [ ]:
portafolio_norm.plot(figsize=(10,6))
plt.title('Portafolio de acciones')
plt.xlabel('Tiempo')
plt.ylabel('Precios')
plt.show()

## Rendimiento de un activo

Cuando se analiza el comportamiento de un activo, es de gran utilidad analizar el comportamiento de los rendimientos generados a través del tiempo.

Vamos a seleccionar un activo y estudiamos sus rendimientos.

In [ ]:
MSFT = portafolio['MSFT']

A partir del método **shift** obtenemos el rendimiento con la siguiente fórmula:
$$
return = \frac {P_t}{P_{t-1}} -1
$$

In [ ]:
MSFT_rendimientos = (MSFT/MSFT.shift(1))-1

In [ ]:
MSFT_rendimientos

Una vez obtenido la serie de rendimientos generados por el activo, realizamos un gráfico de serie con los rendimientos.

In [ ]:
MSFT_rendimientos.plot(figsize=(6,3))
plt.show()

Realizamos un gráfico de barras (histograma).

In [ ]:
plt.hist(MSFT_rendimientos, bins=20)
plt.show()

Note que la distribución de los rendimientos es muy parecido a una distribución normal. Vamos a graficarlo junta a una pdf normal.

In [ ]:
# Fit a normal distribution to
# the data:
# mean and standard deviation
mu = np.mean(MSFT_rendimientos)
std= np.std(MSFT_rendimientos) 
 
# Plot the histogram.
plt.hist(MSFT_rendimientos, bins=20, density = True)
 
# Plot the PDF.
#creamos un array con 100 registros que van de -0.2 a 0.2
x = np.linspace(-0.2, 0.2, 100)  
#generamos la función de densidad de la normal con 
#parámetros obtenidos de la serie
p = norm.pdf(x, mu, std)   
plt.plot(x, p)  #graficamos
plt.title("Histograma vs distribución normal")
 
plt.show()

Obtenemos el rendimiento promedio anual. Se multiplica por 250 porque son los dias de negocio que se tienen a lo largo de un año. 

In [ ]:
MSFT_rend_promedio = MSFT_rendimientos.mean()*250
MSFT_rend_promedio

## Riesgo de un activo

Además de analizar el rendimiento que puede tener un activo en un periodo de tiempo, se debe tener una medida de variabilidad o volatilidad de este. A partir de esta medición, se podrá conocer si el activo representa un riesgo o no para el inversionista.

$$
risk = \sqrt{\frac {\sum(r - \bar{r})^2}{n-1}}
$$

Vamos a elegir dos acciones de nuestro portafolio.

In [ ]:
MSFT = portafolio['MSFT']
GOOGLE = portafolio['GOOGL']

Obtenemos sus rendimientos.

In [ ]:
MSFT_rendimientos = (MSFT/MSFT.shift(1))-1
GOOGLE_rendimientos = (GOOGLE/GOOGLE.shift(1))-1

Obtemenos rendimiento promedio anual.

In [ ]:
print('Rendimiento anual MICROSOFT:',round(MSFT_rendimientos.mean()*250,6))
print('Rendimiento anual GOOGLE:',round(GOOGLE_rendimientos.mean()*250,6))

Obtenemos el riesgo promedio anual.

In [ ]:
print('Riesgo anual MICROSOFT:',round(MSFT_rendimientos.std()*250**0.5,6))
print('Riesgo anual GOOGLE:',round(GOOGLE_rendimientos.std()*250**0.5,6))

## Rendimiento de un portafolio

Obtenemos el rendimiento ahora de un portafolio de acciones.

El método shift permite desfasar un DataFrame el número de renglones que deseemos, de acuerdo a la columna de indices.

In [ ]:
portafolio.head(5)

In [ ]:
portafolio.shift(1).head(5)

A partir del método **shift** obtenemos el rendimiento con la siguiente fórmula:
$$
return = \frac {P_t}{P_{t-1}} -1
$$

In [ ]:
rendimientos = (portafolio / portafolio.shift(1))-1
rendimientos.head(10)

Asignamos un vector de pesos, es decir, el porcentaje de inversión en cada activo. \
Recordemos que la suma de los pesos nos debe dar un total de 1.

In [ ]:
pesos = np.array([0.25,0.25,0.25,0.25])

Obtenemos el rendimiento promedio anual de cada acción. Recordemos que no es correcto multiplicar por 250 porque los dias de negocio al año son entre 250 y 251 (no contando dias no comerciales como fin de semana o dias festivos)


In [ ]:
rendimientos_anuales = rendimientos.mean()*250
rendimientos_anuales

A través del método **dot**, obtenemos el producto de las matrices rendimiento y pesos.

In [ ]:
np.dot(rendimientos_anuales, pesos)

In [ ]:
# formato
print(str(round(np.dot(rendimientos_anuales, pesos),5)*100)+'%')

## Riesgo de un portafolio

Supongamos que decidimos invertir la misma cantidad de dinero en las 5 compañias que componen nuestro portafolio.

In [ ]:
pesos = np.array([0.25,0.25,0.25,0.25])

Obtenemos la varianza del portafolio.

In [ ]:
rendimientos.cov()

In [ ]:
var_portafolio = np.dot(pesos.T,np.dot(rendimientos.cov() * 250, pesos))
var_portafolio

Obtenemos el riesgo del portafolio.

In [ ]:
riesgo_portafolio = (np.dot(
    pesos.T,np.dot(rendimientos.cov() * 250, pesos))) ** 0.5
print(round(riesgo_portafolio * 100,6),'%')

## Frontera Eficiente

La frontera eficiente nos permitirá analizar de manera gráfica las combinaciones de portafolios que nos brindan el mayor rendimiento asumiendo el mismo riesgo que otros portafolios.

Obtenemos la matriz de covarianzas entre los activos.

In [ ]:
cov_portafolio = rendimientos.cov() * 250
cov_portafolio

Obtenemos la matriz de correlación.

In [ ]:
corr_portafolio = rendimientos.corr()
corr_portafolio

Guardamos el numero de activos dentro de una variable.

In [ ]:
no_activos = len(tickers)
no_activos

Recordemos las formulas para obtener el rendimiento, varianza y riesgo esperado del portafolio.

In [ ]:
# Asignamos pesos de manera aleatorio
pesos = np.random.random(no_activos)
pesos = pesos / np.sum(pesos)
print('Porcentaje invertido en cada activo:', pesos)

# Rendimiento esperado del portafolio
rendimiento_port = np.dot(rendimientos_anuales, pesos)
print('Rendimiento esperado:', rendimiento_port * 100)

# Varianza esperada del portafolio
varianza_port = np.dot(pesos.T, np.dot(cov_portafolio,pesos))
print('Varianza esperada:', varianza_port * 100)

# Riesgo esperado del portafolio
riesgo_port = np.sqrt(varianza_port)
print('Riesgo esperado:', riesgo_port * 100)

Construimos diferentes portafolios (con pesos asignados de manera aleatoria) para graficar la froneta eficiente.

In [ ]:
rend_portafolio = []
riesgo_portafolio = []

for x in range(10000):
    pesos = np.random.random(no_activos)
    pesos = pesos / np.sum(pesos)
    rendimiento_port = np.dot(rendimientos_anuales, pesos)
    riesgo_port = np.sqrt(np.dot(pesos.T, np.dot(cov_portafolio,pesos)))

    rend_portafolio.append(rendimiento_port)
    riesgo_portafolio.append(riesgo_port)

rend_portafolio = np.array(rend_portafolio)
riesgo_portafolio = np.array(riesgo_portafolio)

portafolio = pd.DataFrame({'Rendimiento':rend_portafolio,'Riesgo':riesgo_portafolio})

plt.figure(figsize = (10,6))
portafolio.plot(x = 'Riesgo', y = 'Rendimiento', kind = 'scatter', marker = 'o', s = 10, alpha = 0.3, grid = True)
plt.show()

In [ ]:
portafolio

## Portafolio de mínima varianza

A partir de esta teoria de riesgo-rendimiento, podemos utilizar los módulos de optimización para encontrar la cartera que me represente el menor riesgo posible.  

In [ ]:
from scipy.optimize import minimize

def min_var(rendimientos):

    def var_portafolio(pesos):
        cov = rendimientos.cov() * 250
        riesgo_port = np.sqrt(np.dot(pesos.T, np.dot(cov,pesos)))
        return riesgo_port

    def restriccion_pesos(pesos):
        return np.sum(pesos) - 1

    limites = [(0,1),(0,1),(0,1),(0,1)]
    inicial = [0.25, 0.25, 0.25, 0.25]
    restricciones = {'type': 'eq', 'fun':restriccion_pesos}

    optimizar = minimize(fun = var_portafolio,
                         x0 = inicial,
                         bounds = limites,
                         constraints = restricciones,
                         method = 'SLSQP'
                        )

    return list(optimizar['x'])
        

In [ ]:
min_var(rendimientos)

# El modelo CAPM (Capital Asset Pricing Model)

## La fórmula del CAPM

$$
r_{i} = r_{f} + \beta_{im}*(r_{m}-r_{f})
$$

donde:\
$r_i = $ rendimiento esperado del activo\
$r_f = $ rendimiento activo libre de riesgo\
$\beta_{im} = $ beta del activo\
$r_m = $ rendimiento del mercado

El **rendimiento libre de riesgo** es el rendimiento mínimo que un inversor estaria dispuesto a aceptar. 

La diferencia entre $r_{m}$ y $r_{f}$ es la compensación que un inversor recibe por el riesgo aceptado (**equity risk premium**)

La **beta** mide la cantidad de riesgo que conlleva un activo respecto al mercado.

## Calculamos la beta del activo

$$
\beta_{WMT} = \frac {COV_{AMZN,m}}{VAR_{m}}
$$

In [ ]:
tickers = ['AMZN', '^IXIC', '^TNX']

capm = pd.DataFrame()

for t in tickers:
    capm[t] = yf.download(t, start = '2019-10-29', end = '2024-10-29')['Close']

In [ ]:
capm

Obtenemos los rendimientos de nuestro activo y del índice de mercado.

In [ ]:
capm_rendimientos = capm / capm.shift(1) -1

In [ ]:
capm_rendimientos

Para obtener la beta, necesitamos tanto la covarianza entre los rendimientos del activo y el indice, como la varianza de los rendimientos del indice.

In [ ]:
cov = capm_rendimientos.cov() * 250
cov

In [ ]:
#Covarianza activo y mercado
cov_con_mercado = cov.iloc[0,1]
cov_con_mercado

In [ ]:
#Varianza del mercado
var_mercado = capm_rendimientos['^IXIC'].var() * 250
var_mercado

Calculamos la beta.

In [ ]:
AMZN_beta = cov_con_mercado / var_mercado
AMZN_beta

Validamos nuestro calculo con la información obtenida directamente de Yahoo finance.

In [ ]:
activo = yf.Ticker('AMZN')
activo.info.get('beta')

## estimación a través del CAPM

In [ ]:
risk_free = capm.loc['2024-10-28','^TNX'] / 100
risk_free

In [ ]:
market_return = capm_rendimientos['^IXIC'].mean() * 250
market_return

In [ ]:
risk_premium = market_return - risk_free
risk_premium

In [ ]:
amazon_capm_return = risk_free + AMZN_beta * risk_premium
amazon_capm_return

In [ ]:
# estimación a través de promedios
capm_rendimientos['AMZN'].mean() * 250

### Sharpe Ratio

$$
Sharpe \space Ratio = \frac {r_{AMZN} - r_{f}}{\sigma_{AMZN}}
$$

Buscamos  SR > 1

In [ ]:
rendimientos_AMZN = capm_rendimientos['AMZN'].mean() * 250
riesgo_AMZN = capm_rendimientos['AMZN'].std() * 250 ** 0.5

sharpe_AMZN = (rendimientos_AMZN - risk_free) / riesgo_AMZN

In [ ]:
sharpe_AMZN